In [4]:
import os
os.listdir()

['.ipynb_checkpoints',
 '.jupyter',
 'bottom_ranked_by_marketcap_with_esg.csv',
 'data.csv',
 'new_project_ao.ipynb',
 'ranked_by_marketcap_with_esg.csv',
 'top_company_shortlist.csv']

In [5]:
import pandas as pd
df = pd.read_csv("data.csv",usecols=['ticker','name','total_score','total_grade', 'industry'])

In [6]:
df.columns

Index(['ticker', 'name', 'industry', 'total_score', 'total_grade'], dtype='object')

In [7]:
df['industry']

0                                   Media
1                             Automobiles
2      Trading Companies and Distributors
3                       Consumer products
4                                   Media
                      ...                
717                             Machinery
718                           Real Estate
719                            Technology
720                               Energy 
721                   Aerospace & Defense
Name: industry, Length: 722, dtype: object

In [50]:
df['total_score'].describe()

count     722.000000
mean      975.750693
std       218.751796
min       600.000000
25%       763.000000
50%      1046.000000
75%      1144.000000
max      1536.000000
Name: total_score, dtype: float64

In [51]:
df['total_grade'].value_counts()


total_grade
BBB    368
B      167
BB     104
A       83
Name: count, dtype: int64

In [36]:
#filtering top ESG companies
score_threshold = df['total_score'].median()
acceptable_grades = ["A","AA","AAA"]

df_high_esg = df[
    (df["total_score"] >= score_threshold) &
    (df["total_grade"].isin(acceptable_grades))
].copy()

df_high_esg_sorted = df_high_esg.sort_values(by="total_score",ascending=False)

# df_high_esg_sorted[["ticker","name","industry","total_score","total_grade"]]
df_high_esg_sorted.shape[0]

83

In [37]:
# Filtering lowest ESG companies
score_threshold = df['total_score'].median()
unacceptable_grades = ["B", "BB", "BBB", "CCC", "CC", "C", "D"] 

df_low_esg = df[
    (df["total_score"] < score_threshold) &
    (df["total_grade"].isin(unacceptable_grades))
].copy()

df_low_esg_sorted = df_low_esg.sort_values(by="total_score", ascending=True)
# Optional: View count or preview
# df_low_esg_sorted[["ticker", "name", "industry", "total_score", "total_grade"]]
# df_low_esg_sorted.shape[0]


In [10]:
import yfinance as yf

def get_financials(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            "Ticker": ticker,
            "Sector": info.get("sector"),
            "marketCap": info.get("marketCap"),
            "Current Ratio": info.get("currentRatio"),
            "Quick Ratio": info.get("quickRatio"),
            "Debt to Equity": info.get("debtToEquity"),
            "Operating Margin": info.get("operatingMargins"),
            "Profit Margin": info.get("profitMargins"),
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return {"ticker": ticker}

In [39]:
top_esg_tickers = df_high_esg_sorted["ticker"]
top_financial_data = [get_financials(ticker) for ticker in top_esg_tickers]

In [40]:
bottom_esg_tickers = df_low_esg_sorted["ticker"]
bottom_financial_data = [get_financials(ticker) for ticker in bottom_esg_tickers]

HTTP Error 404: 
HTTP Error 404: 
HTTP Error 404: 
HTTP Error 404: 
HTTP Error 404: 
HTTP Error 404: 


In [66]:
financials_df = pd.DataFrame(top_financial_data)
bottom_financial_df = pd.DataFrame(bottom_financial_data)

In [67]:
bottom_sorted_by_mc = bottom_financial_df.sort_values(by="marketCap", ascending=False)
df_sorted_by_mc = financials_df.sort_values(by="marketCap",ascending=False)

In [68]:
# Optional preview
df_sorted_by_mc = df_sorted_by_mc.dropna() #drop rows with nulls
bottom_sorted_by_mc = bottom_sorted_by_mc.dropna()
bottom_sorted_by_mc

,Ticker,Sector,marketCap,Current Ratio,Quick Ratio,Debt to Equity,Operating Margin,Profit Margin
270,nvda,Technology,4.347333e+12,3.388,2.857,12.267,0.49108,0.51694
260,aapl,Technology,3.011414e+12,0.868,0.724,154.486,0.29991,0.24296
347,googl,Communication Services,2.357863e+12,1.904,1.720,11.481,0.32429,0.31118
70,meta,Communication Services,1.917919e+12,1.973,1.706,25.406,0.43019,0.39992
33,avgo,Technology,1.377787e+12,1.077,0.887,166.032,0.31765,0.22643
...,...,...,...,...,...,...,...,...
197,aemd,Healthcare,2.412984e+06,3.133,2.896,16.160,0.00000,0.00000
150,adtx,Healthcare,2.184145e+06,0.080,0.076,25.836,-5508.95360,0.00000
24,utrs,Healthcare,8.880000e+02,1.882,0.880,215.014,-0.50949,-0.64709
341,mtem,Healthcare,6.580000e+02,1.565,1.361,155.070,-14.50350,-0.61351


In [54]:
df_sorted_by_mc["Sector"].unique()

array(['Technology', 'Consumer Cyclical', 'Consumer Defensive',
       'Industrials', 'Healthcare', 'Utilities', 'Energy', 'Real Estate',
       'Basic Materials', 'Financial Services'], dtype=object)

In [55]:
industry_bench = {
    "Technology": {
        "profit_margin": 0.21,            # ~21% net margins for Info Tech sector :contentReference[oaicite:1]{index=1}
        "operating_margin": 0.21,
        "debt_to_equity": 0.65,           # typical IT range ~0.37‑0.92 :contentReference[oaicite:2]{index=2}
        "current_ratio": 2.0              # tech current ratio ~1.5‑3 :contentReference[oaicite:3]{index=3}
    },
    "Consumer Cyclical": {
        "profit_margin": 0.077,           # ~7.7% operating margin :contentReference[oaicite:4]{index=4}
        "operating_margin": 0.077,
        "debt_to_equity": 1.0,            # D/E typical range up to ~3.46 :contentReference[oaicite:5]{index=5}
        "current_ratio": 1.6              # retail & discretionary ~1.2‑1.8 :contentReference[oaicite:6]{index=6}
    },
    "Consumer Defensive": {
        "profit_margin": 0.069,           # consumer staples ~6.9% :contentReference[oaicite:7]{index=7}
        "operating_margin": 0.069,
        "debt_to_equity": 1.0,
        "current_ratio": 1.5
    },
    "Industrials": {
        "profit_margin": 0.106,           # industrials ~10.6% operating margin :contentReference[oaicite:8]{index=8}
        "operating_margin": 0.106,
        "debt_to_equity": 1.0,            # broad industrial range 0.28‑2.18 :contentReference[oaicite:9]{index=9}
        "current_ratio": 1.9              # industrials ~1.5‑2.4 :contentReference[oaicite:10]{index=10}
    },
    "Healthcare": {
        "profit_margin": 0.074,           # healthcare overall ~7.4% :contentReference[oaicite:11]{index=11}
        "operating_margin": 0.074,
        "debt_to_equity": 1.0,            # healthcare D/E ~0.53‑2.82 :contentReference[oaicite:12]{index=12}
        "current_ratio": 2.0              # typical ~1.5‑3 :contentReference[oaicite:13]{index=13}
    },
    "Utilities": {
        "profit_margin": 0.141,           # utilities net margins ~14.1% :contentReference[oaicite:14]{index=14}
        "operating_margin": 0.141,
        "debt_to_equity": 2.0,            # utilities often have high leverage ~1.08‑3.13 :contentReference[oaicite:15]{index=15}
        "current_ratio": 1.8              # utilities liquidity ~1.5‑2.0 :contentReference[oaicite:16]{index=16}
    },
    "Energy": {
        "profit_margin": 0.123,           # oil‑gas integrated ~12.3% :contentReference[oaicite:17]{index=17}
        "operating_margin": 0.123,
        "debt_to_equity": 1.3,            # energy D/E range ~0.29‑2.42 :contentReference[oaicite:18]{index=18}
        "current_ratio": 1.6              # capital‑intensive typical ~1.4‑2.0 :contentReference[oaicite:19]{index=19}
    },
    "Real Estate": {
        "profit_margin": 0.239,           # REIT net margin ~23.9% :contentReference[oaicite:20]{index=20}
        "operating_margin": 0.239,
        "debt_to_equity": 1.0,            # typical REIT ~0.54‑1.57 :contentReference[oaicite:21]{index=21}
        "current_ratio": 1.4              # REITs ~1.2‑1.6 :contentReference[oaicite:22]{index=22}
    },
    "Basic Materials": {
        "profit_margin": 0.101,           # materials ~10.1% operating profit margin :contentReference[oaicite:23]{index=23}
        "operating_margin": 0.101,
        "debt_to_equity": 0.9,            # materials D/E ~0.2‑1.29 :contentReference[oaicite:24]{index=24}
        "current_ratio": 1.8              # base materials ~1.6‑2.2 :contentReference[oaicite:25]{index=25}
    },
    "Financial Services": {
        "profit_margin": 0.135,           # non‑bank financial services ~13.5% :contentReference[oaicite:26]{index=26}
        "operating_margin": 0.135,
        "debt_to_equity": 2.0,            # financials up to ~4.5 but average ~2.0 :contentReference[oaicite:27]{index=27}
        "current_ratio": 1.5              # banks often around 1.3‑2.0 :contentReference[oaicite:28]{index=28}
    },
    "Communication Services": {
    "profit_margin": 0.125,            # Net profit margin ≈ 12.5% for telecom/communication services sector :contentReference[oaicite:1]{index=1}
    "operating_margin": 0.195,         # Operating margin roughly 19.5% for telecommunications services :contentReference[oaicite:2]{index=2}
    "debt_to_equity": 1.89,            # Average debt-to-equity ratio ≈ 1.89 for Communication Services industry :contentReference[oaicite:3]{index=3}
    "current_ratio": 1.30              # Typical current ratio ~1.3 for telecom services companies :contentReference[oaicite:4]{index=4}
    }
}

In [56]:
def score_ratio(company, benchmark, higher_higher=True):
    if company is None or pd.isna(company) or benchmark is None or pd.isna(benchmark):
        return 0.0

    if benchmark == 0 or company == 0:
        return 0.0

    # Directional ratio
    ratio = company / benchmark if higher_higher else benchmark / company

    # Cap ratio to a reasonable range to avoid overflow
    ratio = max(0.01, min(ratio, 100))  # You can tune this range if needed

    # Smoothed sigmoid scoring
    score = 1 / (1 + np.exp(-2 * (ratio - 1)))
    return round(score, 4)

In [57]:
def calculate_scores(row):
    sector = row['Sector']
    if sector not in industry_bench:
        return pd.Series({
            'liquidity_score': 0,
            'solvency_score': 0,
            'op_margin_score': 0,
            'profit_margin_score': 0,
            'profit_score': 0,
            'financial_health_score': 0
        })

    bench = industry_bench[sector]

    # Financial health indicators
    liquidity_score = score_ratio(row['Current Ratio'], bench['current_ratio'])
    solvency_score = score_ratio(row['Debt to Equity'], bench['debt_to_equity'], higher_higher=False)

    # Profitability indicators
    op_margin_score = score_ratio(row['Operating Margin'], bench['operating_margin'])
    profit_margin_score = score_ratio(row['Profit Margin'], bench['profit_margin'])

    # Composite Scores
    profit_score = round((op_margin_score + profit_margin_score) / 2, 4)
    financial_health_score = round(
        0.25 * liquidity_score +
        0.25 * solvency_score +
        0.5 * profit_score,
        4
    )

    return pd.Series({
        'liquidity_score': liquidity_score,
        'solvency_score': solvency_score,
        'op_margin_score': op_margin_score,
        'profit_margin_score': profit_margin_score,
        'profit_score': profit_score,
        'financial_health_score': financial_health_score
    })


In [69]:
final_df_top = df_sorted_by_mc.copy()
final_df_top = final_df_top.join(final_df_top.apply(calculate_scores, axis=1))

In [70]:
final_df_bottom = bottom_sorted_by_mc.copy()
final_df_bottom = final_df_bottom.join(final_df_bottom.apply(calculate_scores, axis=1))

In [72]:
final_df_bottom

,Ticker,Sector,marketCap,Current Ratio,Quick Ratio,Debt to Equity,Operating Margin,Profit Margin,liquidity_score,solvency_score,op_margin_score,profit_margin_score,profit_score,financial_health_score
270,nvda,Technology,4.347333e+12,3.388,2.857,12.267,0.49108,0.51694,0.8003,0.1308,0.9357,0.9490,0.9424,0.7040
260,aapl,Technology,3.011414e+12,0.868,0.724,154.486,0.29991,0.24296,0.2438,0.1213,0.7019,0.5778,0.6398,0.4112
347,googl,Communication Services,2.357863e+12,1.904,1.720,11.481,0.32429,0.31118,0.7169,0.1583,0.7902,0.9516,0.8709,0.6542
70,meta,Communication Services,1.917919e+12,1.973,1.706,25.406,0.43019,0.39992,0.7380,0.1357,0.9178,0.9879,0.9528,0.6948
33,avgo,Technology,1.377787e+12,1.077,0.887,166.032,0.31765,0.22643,0.2843,0.1213,0.7360,0.5390,0.6375,0.4202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,aemd,Healthcare,2.412984e+06,3.133,2.896,16.160,0.00000,0.00000,0.7564,0.1328,0.0000,0.0000,0.0000,0.2223
150,adtx,Healthcare,2.184145e+06,0.080,0.076,25.836,-5508.95360,0.00000,0.1279,0.1276,0.1213,0.0000,0.0606,0.0942
24,utrs,Healthcare,8.880000e+02,1.882,0.880,215.014,-0.50949,-0.64709,0.4705,0.1213,0.1213,0.1213,0.1213,0.2086
341,mtem,Healthcare,6.580000e+02,1.565,1.361,155.070,-14.50350,-0.61351,0.3929,0.1213,0.1213,0.1213,0.1213,0.1892


In [73]:
# Merge ESG data (on Ticker)
final_esg_mc_df = final_df_top[["Ticker", "Sector", "marketCap", "financial_health_score"]].merge(df[['ticker','name','total_score','total_grade']], left_on="Ticker", right_on="ticker",
                                                                                                     how="left")
# Assign Rank based on marketCap (already sorted)
final_esg_mc_df = final_esg_mc_df.reset_index(drop=True)
final_esg_mc_df["Rank"] = final_esg_mc_df.index + 1

# Reorder columns for PowerBI
cols = ["Rank", "Ticker", "name", "Sector", "marketCap", "total_score", "total_grade", "financial_health_score"]
final_esg_mc_df = final_esg_mc_df[cols]


In [74]:
# Merge ESG data (on Ticker)
final_bottom_esg_mc_df = final_df_bottom[["Ticker", "Sector", "marketCap", "financial_health_score"]].merge(df[['ticker','name','total_score','total_grade']], left_on="Ticker", right_on="ticker",
                                                                                                     how="left")

# Assign Rank based on marketCap (already sorted)
final_bottom_esg_mc_df = final_bottom_esg_mc_df.reset_index(drop=True)
final_bottom_esg_mc_df["Rank"] = final_bottom_esg_mc_df.index + 1

# Reorder columns for PowerBI
cols = ["Rank", "Ticker", "name", "Sector", "marketCap", "total_score", "total_grade", "financial_health_score"]
final_bottom_esg_mc_df = final_bottom_esg_mc_df[cols]


In [75]:
final_bottom_esg_mc_df.to_csv("bottom_ranked_by_marketcap_with_esg.csv", index=False)

In [76]:
# Export to CSV
final_esg_mc_df.to_csv("ranked_by_marketcap_with_esg.csv", index=False)

In [77]:
final_bottom_esg_mc_df

,Rank,Ticker,name,Sector,marketCap,total_score,total_grade,financial_health_score
0,1,nvda,NVIDIA Corp,Technology,4.347333e+12,899,BB,0.7040
1,2,aapl,Apple Inc,Technology,3.011414e+12,891,BB,0.4112
2,3,googl,Alphabet Inc,Communication Services,2.357863e+12,1012,BBB,0.6542
3,4,meta,Meta Platforms Inc,Communication Services,1.917919e+12,652,B,0.6948
4,5,avgo,Broadcom Inc,Technology,1.377787e+12,622,B,0.4202
...,...,...,...,...,...,...,...,...
232,233,aemd,Aethlon Medical Inc,Healthcare,2.412984e+06,777,BB,0.2223
233,234,adtx,Aditxt Inc,Healthcare,2.184145e+06,739,B,0.0942
234,235,utrs,Minerva Surgical Inc,Healthcare,8.880000e+02,616,B,0.2086
235,236,mtem,Molecular Templates Inc,Healthcare,6.580000e+02,1005,BBB,0.1892
